# =============================================================
# MILESTONE 4: MLOps, Deployment, and Monitoring
# =============================================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from pathlib import Path
import joblib
import shap
import requests
import json
import time
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import ks_2samp
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
PEACH      = '#FFCBA4'
PEACH_DARK = '#FF9A76'
SAGE       = '#A8C686'
SAGE_DARK  = '#7A9B57'
NEUTRAL    = '#F5F5DC'
ACCENT     = '#E07B39'

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

In [ ]:
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
MODELS_DIR   = PROJECT_ROOT / "models" / "trained_models"
ARTIFACTS_DIR = PROJECT_ROOT / "models" / "artifacts"
DATA_PATH    = PROJECT_ROOT / "data" / "processed" / "final_processed_data.csv"
VIZ_STATIC   = PROJECT_ROOT / "visualizations" / "static" 
VIZ_INTER    = PROJECT_ROOT / "visualizations" / "interactive"

In [ ]:
print("\n[1/8] Loading production artifacts...")
best_model = joblib.load(MODELS_DIR / "best_churn_model.pkl")
selected_features = joblib.load(ARTIFACTS_DIR / "selected_features.pkl")
X_test, y_test = joblib.load(ARTIFACTS_DIR / "test_data.pkl")

df_full = pd.read_csv(DATA_PATH)
X_full = df_full.drop('Churn', axis=1)
y_full = df_full['Churn'].astype(int)

print(f"Model: {type(best_model).__name__}")
print(f"Features: {len(selected_features)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
print("\n[2/8] Computing SHAP values...")
explainer = shap.TreeExplainer(best_model)
X_sample = X_test.sample(n=min(500, len(X_test)), random_state=42)

shap_values = explainer.shap_values(X_sample)

# ---- FIX: Ensure shap_vals is always (samples, features) ----
if isinstance(shap_values, list):
    shap_vals = shap_values[1]
elif shap_values.ndim == 3:
    shap_vals = shap_values[1]
else:
    shap_vals = shap_values

print("shap_vals shape:", shap_vals.shape)

# Align feature names if needed
if len(selected_features) != shap_vals.shape[1]:
    print("WARNING: feature count mismatch → fixing automatically")
    selected_features = selected_features[:shap_vals.shape[1]]

# Global Feature Importance
shap_importance = pd.DataFrame({
    'Feature': selected_features,
    'Mean_|SHAP|': np.abs(shap_vals).mean(axis=0)
}).sort_values('Mean_|SHAP|', ascending=False)


In [ ]:
print("\n[3/8] Testing FastAPI endpoint...")
API_URL = "http://127.0.0.1:8000/predict"

sample_payload = {
    "Account length": 50,
    "International plan": "Yes",
    "Voice mail plan": "No",
    "Number vmail messages": 0,
    "Total day minutes": 300.0,
    "Customer service calls": 5
}

try:
    resp = requests.post(API_URL, json=sample_payload, timeout=10)
    if resp.status_code == 200:
        result = resp.json()
        print("API CONNECTED & WORKING")
        print(f"→ Churn Probability: {result['churn_probability']:.2%}")
        print(f"→ Top Driver: {result['top_shap_features'][0]['feature']} ({result['top_shap_features'][0]['shap_value']:+.3f})")
    else:
        print(f"API Error {resp.status_code}: {resp.text}")
except Exception as e:
    print("API not reachable. Start with: uvicorn src.api.main:app --reload")
    print(e)

In [ ]:
print("\n[4/8] Performance benchmarking...")
sizes = [1, 10, 50, 100, 500]
bench = []
for size in sizes:
    batch = X_test.sample(n=min(size, len(X_test)), random_state=42)
    start = time.time()
    _ = best_model.predict_proba(batch)
    elapsed = (time.time() - start) * 1000
    bench.append({'Size': size, 'Latency_ms': elapsed, 'Per_Sample_ms': elapsed/size})

bench_df = pd.DataFrame(bench)
fig = make_subplots(1, 2, subplot_titles=("Latency per Sample", "Throughput"))
fig.add_trace(go.Scatter(x=bench_df['Size'], y=bench_df['Per_Sample_ms'], mode='lines+markers',
                         name='Latency/Sample', line=dict(color=PEACH_DARK)), row=1, col=1)
fig.add_trace(go.Scatter(x=bench_df['Size'], y=bench_df['Size']/(bench_df['Latency_ms']/1000),
                         mode='lines+markers', name='Throughput', line=dict(color=SAGE_DARK)), row=1, col=2)
fig.update_layout(title="Model Inference Performance", height=400, template="plotly_white")
fig.write_html(VIZ_INTER / "04_latency_throughput.html")
fig.show()

In [ ]:
print("\n[5/8] Data drift analysis...")
train_split = int(0.8 * len(X_full))
X_train_ref = X_full.iloc[:train_split]
X_prod = X_full.iloc[train_split:]

drift_results = []
for feat in selected_features:
    stat, p = ks_2samp(X_train_ref[feat], X_prod[feat])
    drift_results.append({'Feature': feat, 'KS': stat, 'p_value': p, 'Drift': p < 0.05})

drift_df = pd.DataFrame(drift_results).sort_values('KS', ascending=False)
drifted_count = drift_df['Drift'].sum()
print(f"{drifted_count}/{len(selected_features)} features show significant drift")

fig = px.bar(drift_df.head(15), x='KS', y='Feature', orientation='h',
             color='Drift', color_discrete_map={True: PEACH_DARK, False: SAGE_DARK},
             title="Top 15 Features by Data Drift (KS Test)")
fig.update_layout(yaxis=dict(autorange="reversed"), height=600)
fig.write_html(VIZ_INTER / "05_data_drift.html")
fig.show()

In [ ]:
print("\n[6/8] Business impact analysis...")
proba = best_model.predict_proba(X_test)[:, 1]
risk_df = pd.DataFrame({'prob': proba, 'actual': y_test.values})
risk_df['segment'] = pd.cut(proba, [0, 0.3, 0.7, 1.0], labels=['Low', 'Medium', 'High'])

segment_stats = risk_df.groupby('segment').agg(
    count=('prob', 'size'),
    avg_prob=('prob', 'mean'),
    churn_rate=('actual', 'mean')
).round(4)

high_risk_n = segment_stats.loc['High', 'count']
revenue_at_risk = high_risk_n * 1200  # $1200 annual value
campaign_cost = high_risk_n * 200
success_rate = 0.40
retained_value = high_risk_n * success_rate * 1200
roi = (retained_value - campaign_cost) / campaign_cost * 100

fig = px.histogram(risk_df, x='prob', color='segment',
                   color_discrete_map={'Low': SAGE_DARK, 'Medium': PEACH, 'High': PEACH_DARK},
                   title="Customer Distribution by Predicted Churn Risk")
fig.write_html(VIZ_INTER / "06_risk_segments.html")
fig.show()